# Customer Analysis - Explore Customer Behavior

## Import

Needed packages. Pyspark will be used for data managing and plotly for visualisations. Keep in mind to install
JAVA so Spark will work properly.

Used dataset is from https://rees46.com/de found on https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store.

In [2]:
import os
import pyspark
import pandas as pd
import pyspark.sql.functions as f
import plotly.express as px
import plotly.graph_objects as go

## Read

The data needs to be located in ```data/``` in unzipped form as a csv.

In [3]:
# read raw data
spark = pyspark.sql.SparkSession.builder.appName("app1").getOrCreate()
# sdf = spark.read.csv("data/*.csv", header=True, inferSchema=True)
# sdf_201911 = spark.read.csv("data/2019-Nov.csv", header=True, inferSchema=True)
# sdf_201910 = spark.read.csv("data/2019-Oct.csv", header=True, inferSchema=True)

In [4]:
# join both months together
# sdf = sdf_201910.union(sdf_201911)
sdf = spark.read.csv("data/test_data.csv", header=True, inferSchema=True)
sdf.show()

+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 00:00:...|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:...|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:...|      view|  17302664|2053013553853497655|                null|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:...|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:...|      view|   1004775|2053013555631882655|elect

## Preparation

Prepare and enhance data for analysis and modelling.

In [5]:
# Datatypes
sdf = sdf.withColumn("event_time", sdf["event_time"].cast(pyspark.sql.types.TimestampType()))
sdf = sdf.withColumn("category_id", sdf["category_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("product_id", sdf["product_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("user_id", sdf["user_id"].cast(pyspark.sql.types.StringType()))

# Feature Splitting
# sdf = sdf.withColumn("category_class", f.substring_index(sdf.category_code, '.', 1))

sdf = sdf.withColumn("category_class", f.split(sdf["category_code"], "\.").getItem(0))
sdf = sdf.withColumn("category_sub_class", f.split(sdf["category_code"], "\.").getItem(1))
sdf = sdf.withColumn("category_sub_sub_class", f.split(sdf["category_code"], "\.").getItem(2))

sdf = sdf.withColumn("year", f.year("event_time"))
sdf = sdf.withColumn("month", f.month("event_time"))
sdf = sdf.withColumn("weekofyear", f.weekofyear("event_time"))
sdf = sdf.withColumn("dayofyear", f.dayofyear("event_time"))
sdf = sdf.withColumn("dayofweek", f.dayofweek("event_time"))
sdf = sdf.withColumn("dayofmonth", f.dayofmonth("event_time"))
sdf = sdf.withColumn("hour", f.hour("event_time"))

sdf = sdf.withColumn('turnover', f.when(f.col('event_type') == 'purchase', f.col('price')).otherwise(0))
sdf = sdf.withColumn('bougth_quantity', f.when(f.col('event_type') == 'purchase', f.lit(1)).otherwise(0))
sdf = sdf.withColumn('viewed_quantity', f.when(f.col('event_type') == 'view', f.lit(1)).otherwise(0))
sdf = sdf.withColumn('cart_quantity', f.when(f.col('event_type') == 'cart', f.lit(1)).otherwise(0))
# None Handling
# sdf = sdf.fillna(value="not defined")

sdf.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)
 |-- category_class: string (nullable = true)
 |-- category_sub_class: string (nullable = true)
 |-- category_sub_sub_class: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- dayofyear: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- turnover: double (nullable = true)
 |-- bougth_quantity: integer (nullable = false)
 |-- viewed_quantity: integer (nullable = false)
 |-- cart_quantity: integer (nullable = false)



## Dataframe Creation

create several dataframes with different aggregation level to answer different questions/ tasks.

In [6]:
# raw
sdf_raw = sdf

In [7]:
sdf.createOrReplaceTempView("Data")

In [8]:
# aggregated customer
sdf_agg_cust = sdf.groupBy("user_id", "user_session", "event_type", "product_id").mean("price")


In [9]:
# aggregated session

In [10]:
# aggregated product


In [11]:
# aggregated class

In [12]:
# aggregated time (weeks, dayofweeks, month)

sdf_time_dist_month = sdf.groupBy("event_type", "dayofmonth").count()
sdf_time_dist_month = sdf_time_dist_month.withColumnRenamed("count", "cnt")
sdf_time_dist_month = sdf_time_dist_month.sort("dayofmonth", "event_type")


In [13]:
sdf_time_dist_week = sdf.groupBy("event_type", "dayofweek").count()
sdf_time_dist_week = sdf_time_dist_week.withColumnRenamed("count", "cnt")
sdf_time_dist_week = sdf_time_dist_week.sort("dayofweek", "event_type")


In [14]:
sdf_time_dist_day = sdf.groupBy("event_type", "hour").count()
sdf_time_dist_day = sdf_time_dist_day.withColumnRenamed("count", "cnt")
sdf_time_dist_day = sdf_time_dist_day.sort("hour", "event_type")


## Field Explanations

Following fields are in the standard dataset:
- event_time
- product_id
- category_id
- category_code
- brand
- price
- user_id
- user_session

### General
In this overview you can see the count of unique rows, product_ids, category_classes, category_codes, category_ids, brands, user_ids and user_sessions as well as the average price of the products.

In [15]:
sdf_count_overview = spark.sql("SELECT COUNT(*) AS Row_Count, \
                                       COUNT(DISTINCT(product_id)) AS Product_ID, \
                                       COUNT(DISTINCT(category_class)) AS Category_Class, \
                                       COUNT(DISTINCT(category_code)) AS Category_Code, \
                                       COUNT(DISTINCT(category_id)) AS Category_ID, \
                                       COUNT(DISTINCT(brand)) AS Brand, \
                                       COUNT(DISTINCT(user_id)) AS User_ID, \
                                       COUNT(DISTINCT(user_session)) AS User_Session, \
                                       ROUND(MEAN(price),2) AS AVG_Price \
                                FROM Data")
sdf_count_overview.show()

+---------+----------+--------------+-------------+-----------+-----+-------+------------+---------+
|Row_Count|Product_ID|Category_Class|Category_Code|Category_ID|Brand|User_ID|User_Session|AVG_Price|
+---------+----------+--------------+-------------+-----------+-----+-------+------------+---------+
|      244|       214|             9|           41|         90|   98|    153|         153|   302.15|
+---------+----------+--------------+-------------+-----------+-----+-------+------------+---------+



In [16]:
sdf_raw.show(1, vertical=True)
print(f"Number of total rows: {sdf_raw.count()}")

-RECORD 0--------------------------------------
 event_time             | 2019-11-01 01:00:00  
 event_type             | view                 
 product_id             | 1003461              
 category_id            | 2053013555631882655  
 category_code          | electronics.smart... 
 brand                  | xiaomi               
 price                  | 489.07               
 user_id                | 520088904            
 user_session           | 4d3b30da-a5e4-49d... 
 category_class         | electronics          
 category_sub_class     | smartphone           
 category_sub_sub_class | null                 
 year                   | 2019                 
 month                  | 11                   
 weekofyear             | 44                   
 dayofyear              | 305                  
 dayofweek              | 6                    
 dayofmonth             | 1                    
 hour                   | 1                    
 turnover               | 0.0           

### event_time

In [17]:
sdf_raw.select("event_time").show(5)
print(f"Number of distinct event_time rows: {sdf_raw.select('event_time').distinct().count()}")

+-------------------+
|         event_time|
+-------------------+
|2019-11-01 01:00:00|
|2019-11-01 01:00:00|
|2019-11-01 01:00:01|
|2019-11-01 01:00:01|
|2019-11-01 01:00:01|
+-------------------+
only showing top 5 rows

Number of distinct event_time rows: 84


### event_type
The event_type describes the kind of interaction, an user had with a product. The field can take three forms: View, Cart and Purchase. The distribution of these three forms is represented in the following plot:

In [18]:
sdf_event_type_dist = sdf_raw.groupBy("event_type").count()
sdf_event_type_dist.show()

+----------+-----+
|event_type|count|
+----------+-----+
|  purchase|    2|
|      view|  240|
|      cart|    2|
+----------+-----+



In [19]:
# Plot Event Types
df = sdf_event_type_dist.toPandas()
fig = px.pie(df, values='count', names='event_type', title='Distribution of Customer Actions')
fig.show()

### product_id
The product_id is the unique identificator for a product. As you can see in the overview, there are ... unique product_ids in the datasets Oct-2019 and Nov-2019, the users have interacted with.


In [20]:
sdf_count_per_product_id = spark.sql("SELECT DISTINCT(product_id) AS Product_ID, \
                                                COUNT(product_id) AS Count \
                                        FROM Data \
                                        GROUP BY product_id \
                                        ORDER BY Count DESC")
px.bar(sdf_count_per_product_id.limit(10).toPandas(), x='Product_ID', y='Count', title="Top 10 most interacted products")


In [21]:
# TODO BE DELTED -> find error why not equal to above
sdf_product_id_dist = sdf_raw.groupBy("product_id").count().orderBy("count")

px.bar(sdf_product_id_dist.limit(10).toPandas(), x='product_id', y='count', title="Top 10 most interacted products")

### category_id
The category_id is an unique identifier for the category of a Product. Every Product is assigned to a single category_id, which is summarizing many product_ids into categories. This knowledge is based on the more detailed analyzes within the file "product_analysis.ipnynb". As you can see in the overview, there are ... unique category_ids.

In [22]:
sdf_count_per_category_id=spark.sql("SELECT DISTINCT(category_id) AS Category_ID, \
                                            COUNT(*) AS Count \
                                     FROM Data \
                                     GROUP BY category_id \
                                     ORDER BY Count DESC")
                                     
px.bar(sdf_count_per_category_id.limit(10).toPandas(), x="Category_ID", y="Count", title="Top 10  category_ids most interacted with")  

### category_code
The category_code is describing the category, a product_id and category_id is assigned to. Every Product_id and Category_id is assigned to a single category_code, which is summarizing many product_ids and category_ids into categories. This knowledge is also based on the more detailed analyzes within the file "product_analysis.ipnynb". As you can see in the overview, there are ... unique category_code.

In [23]:
sdf_count_per_category_code=spark.sql("SELECT DISTINCT(category_code) AS Category_Code, \
                                              COUNT(product_id) AS Count \
                                        FROM Data \
                                        GROUP BY category_code \
                                        ORDER BY Count DESC")
px.bar(sdf_count_per_category_code.limit(10).toPandas(), x="Category_Code", y="Count", title="Top 10 category_code most interacted with")

In [24]:
sdf_agg_classes = sdf_raw.groupBy("category_class", "category_sub_class", "category_sub_sub_class").count().na.fill(value="not defined")
sdf_agg_classes = sdf_agg_classes.where(sdf_agg_classes["category_class"] != "not defined")
sdf_agg_classes.show()

+--------------+------------------+----------------------+-----+
|category_class|category_sub_class|category_sub_sub_class|count|
+--------------+------------------+----------------------+-----+
|       apparel|             jeans|           not defined|    3|
|          auto|       accessories|            compressor|    2|
|       apparel|             shoes|           not defined|    1|
|  construction|             tools|                 drill|    4|
|   electronics|             audio|             headphone|    5|
|    appliances|           kitchen|               blender|    1|
|     furniture|           kitchen|                 table|    1|
|    appliances|    sewing_machine|           not defined|    2|
|    appliances|           kitchen|         refrigerators|    5|
|   accessories|               bag|           not defined|    1|
|     computers|       peripherals|               printer|    2|
|     furniture|          bathroom|                  bath|    1|
|    appliances|         

### brand
The brand indicates the brand of a product_id. It is independent of the categories, so that a brand can appear in many category_classes. This knowledge is also based on the more detailed analyzes within the file "product_analysis.ipnynb". There are ... unique brands in the dataset. Thereby you can see the most popular brands in the following plot:

In [25]:
sdf_count_per_brand=spark.sql("SELECT DISTINCT(brand) AS Brand, \
                                      COUNT(*) AS Count \
                                FROM Data \
                                GROUP BY brand \
                                ORDER BY Count DESC")

px.histogram(sdf_count_per_brand.limit(10).toPandas(), x="Brand", y="Count", title="Top 10 brands most interacted with")                               

### price

In [26]:
sdf_raw.describe("price").show()

+-------+-----------------+
|summary|            price|
+-------+-----------------+
|  count|              244|
|   mean|302.1450819672131|
| stddev|425.3893301363122|
|    min|             1.09|
|    max|          2496.59|
+-------+-----------------+



In [27]:
px.box(sdf_raw.groupBy("product_id").avg("price").toPandas(), y="avg(price)", title="Price distribution for products in Store")


### user_id

In [28]:
sdf_raw.select("user_id").show(5)

print("Number of users:")
print(sdf_raw.select("user_id").distinct().count())

+---------+
|  user_id|
+---------+
|520088904|
|530496790|
|561587266|
|518085591|
|558856683|
+---------+
only showing top 5 rows

Number of users:
153


### user_session

In [29]:
# avg actions per session
sdf_cnt_action_per_session = sdf_raw.groupby("user_session").count()
sdf_cnt_action_per_session.describe().show()

+-------+--------------------+------------------+
|summary|        user_session|             count|
+-------+--------------------+------------------+
|  count|                 153|               153|
|   mean|                null|1.5947712418300655|
| stddev|                null|1.0287825488662496|
|    min|0110890b-96d6-4ae...|                 1|
|    max|ff868137-fb3e-4da...|                 6|
+-------+--------------------+------------------+



## Feature Enginnering

### category_class and category_sub_class
The category_code consists of two or three parts in general, which are separeted by a dot. A possible category_code is for example: appliances.kitchen.washer or electronics.smartphone. Because of that the category code can be splited into to the categories: category_class, category_sub_class and category_sub_sub_class.

The category_class is representing the fist part of the category_code. It can be used to summarize many category_codes into an overarching category_class. As you can see in the overview, there are ... unique category_classes.

In [30]:
px.sunburst(sdf_agg_classes.toPandas(), path=["category_class", "category_sub_class", "category_sub_sub_class"], values="count", title="Category Classes and Subclasses (without data for class = 'not defined')")

### Time - Fields

The column “event_time” allows you to create the following columns: year, month, weekofyear, dayofyear, dayofweek, dayofmonth and hour. These columns allow advanced analysis.

- Year: The Year-column contains only the year 2019 since the dataset only covers this year.
- Month: The month-column cotains the values 10, 11 and 12,  which are representing the october, november and december 2019.DeprecationWarning
- Weekofyear: The weekofyear-column is covering the weeks ... - ...
- Dayofyear: The dayofyear-column is covering the days .... - ...
- Dayofweek: The dayofweek-column is covering the values 1-7. These values are representing days sunday(1), monday(2), Tuesday(3), Wednesday(4), Thursday(5), Friday(6) and Saturday(7).
- Dayofmonth: The dayofmonth-column contains the values 1-31, which are reperesenting the day in the corresponding month.
- hour: The hour-column contains the values 0-24, which are representing the hour of interaction.

### Turnover and quantities

In the following table you can see that in some cases a product ID was purchased several times per user session.

In [79]:
sdf_count_session_product = spark.sql("SELECT user_session, \
                                                product_id, \
                                                event_type, \
                                                COunt(*) AS Count \
                                        FROM Data \
                                        GROUP BY user_session, product_id, event_type \
                                            HAVING event_Type = 'purchase' \
                                        ORDER BY Count DESC")
sdf_count_session_product.show()

+--------------------+----------+----------+-----+
|        user_session|product_id|event_type|Count|
+--------------------+----------+----------+-----+
|e6b7ce9b-1938-4e2...|   1005161|  purchase|    1|
|d6034fa2-41fb-4ac...|  13200605|  purchase|    1|
+--------------------+----------+----------+-----+



From this it can be concluded that a single interaction is created for each product purchased. With the help of this information, the columns "turnover", "bought_quantity", "viewed_quantity", "cart_quantity" can be created.

- Turnover: The turnover is equivalent to the price, if the event_type is equal to "purchase".
- bought_quantity: The bought_quantity describes the quantity of a product,that had been bought. In the unaggregated form it only contains the values 0 and 1.
- viewed_quantity: The viewed_quantity describes the quantity of a product,that had been viewed. In the unaggregated form it only contains the values 0 and 1.
- cart_quantity: The cart_quantity describes the quantity of a product,that had been put into the cart. In the unaggregated form it only contains the values 0 and 1.

These columns are particularly suitable for aggregated analyses.

In [90]:
spark.sql("SELECT * FROM Data").show()

+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+--------------+------------------+----------------------+----+-----+----------+---------+---------+----------+----+--------+---------------+---------------+-------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|category_class|category_sub_class|category_sub_sub_class|year|month|weekofyear|dayofyear|dayofweek|dayofmonth|hour|turnover|bougth_quantity|viewed_quantity|cart_quantity|
+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+--------------+------------------+----------------------+----+-----+----------+---------+---------+----------+----+--------+---------------+---------------+-------------+
|2019-11-01 01:00:00|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|4

In [98]:
sdf_count_turnover_quantity = spark.sql("SELECT SUM(turnover), SUM(bougth_quantity), SUM(viewed_quantity), SUM(Cart_quantity) \
                                        FROM Data")
sdf_count_turnover_quantity.show()

+-----------------+--------------------+--------------------+------------------+
|    sum(turnover)|sum(bougth_quantity)|sum(viewed_quantity)|sum(Cart_quantity)|
+-----------------+--------------------+--------------------+------------------+
|778.2199999999999|                   2|                 240|                 2|
+-----------------+--------------------+--------------------+------------------+



## Exploration and Analysis

### Time Distribution

#### Time and Events

In [31]:
# Timestamp Distribution (per event_type) over every day of month

df = sdf_time_dist_month.toPandas()

fig = px.bar(df, x = 'dayofmonth', y = 'cnt', color ='event_type', barmode = 'stack')

fig.update_layout(title = "Number of events over a month",
     xaxis_title = 'Day of Month', yaxis_title = 'Number of Events')
fig.update_xaxes(type="category")
fig.show()

In [32]:
# Timestamp Distribution (per event_type) over every day of week

df = sdf_time_dist_week.toPandas()

fig = px.bar(df, x = 'dayofweek', y = 'cnt', color ='event_type', barmode = 'stack')

fig.update_layout(title = "Number of events over a week",
     xaxis_title = 'Day of Week', yaxis_title = 'Number of Events')
fig.update_xaxes(type="category")
fig.show()

In [33]:
# Timestamp Distribution (per event_type) over every hour of a day

df = sdf_time_dist_day.toPandas()

fig = px.bar(df, x = 'hour', y = 'cnt', color ='event_type', barmode = 'stack')

fig.update_layout(title = "Number of events over a day",
     xaxis_title = 'Hour of day', yaxis_title = 'Number of Events')
fig.update_xaxes(type="category")
fig.show()

#### Time and Turnover

In [34]:
sdf_month_Umsatz = spark.sql("SELECT month, \
                                    dayofmonth, \
                                    SUM(turnover) AS Turnover\
                            FROM Data \
                            GROUP BY month, dayofmonth \
                            ORDER BY Turnover DESC")

In [35]:
df = sdf_month_Umsatz.toPandas()
fig = go.Figure()
fig.update_layout(
    template="simple_white",
    xaxis=dict(title_text="Day of Month"),
    yaxis=dict(title_text="Count"),
    barmode="stack",
)

fig.add_trace(
        go.Bar(x=[df.dayofmonth, df.month], y=df.Turnover),
    )
fig

In [36]:
sdf_week_Umsatz = spark.sql("SELECT weekofyear, \
                                    dayofweek, \
                                    SUM(turnover) AS Turnover\
                            FROM Data \
                            GROUP BY weekofyear, dayofweek \
                            ORDER BY Turnover DESC")

In [37]:
df = sdf_week_Umsatz.toPandas()
fig = go.Figure()
fig.update_layout(
    template="simple_white",
    xaxis=dict(title_text="Day of Month"),
    yaxis=dict(title_text="Count"),
    barmode="stack",
)

fig.add_trace(
        go.Bar(x=[df.dayofweek, df.weekofyear], y=df.Turnover),
    )
fig

### Category and products

#### Connection between category_class, category_code, category_id, product_id and brand

Connection between category_class, category_code, category_id, product_id and brand
The product_id is a subset of the category_id, which is a subset of the category_code. The category_code is in turn a subset of the category_class. (product_id ⊂ category_id ⊂ category_code ⊂ category_class). The brand on the otherhand is cross-class. This knowledge is based on the more detailed analyzes within the file "product_analysis.ipnynb".

In the following plot you can see distribution of the product_id, category_id and category_code within the category_class. It´s possible to access a more detailed view by selecting a special category_class, category_code or category_id.

In [38]:
sdf_product_per_category = sdf_raw.groupBy("category_id").agg(f.countDistinct("product_id"))

df = sdf_product_per_category.toPandas()
px.box(df, y="count(product_id)", title="Number of products per category_id")

In [39]:
sdf_agg_brand_category = sdf_raw.groupBy("category_class", "brand", "product_id").count().na.fill(value="not defined")
px.sunburst(sdf_agg_brand_category.toPandas(), path=["category_class", "brand"], values="count", title="Brands per Category_class")

#### Connection to the price

The following plots will represent the price distribution within the category_classes, category_codes, category_ids, product_ids and brands.

In [40]:
px.box(sdf_raw.toPandas(), x="category_class", y="price", title="Price ~ Category_class")

In [41]:
sdf_price_per_product=spark.sql("SELECT DISTINCT(Product_ID), \
                                        Price \
                                 FROM Data \
                                 ORDER BY Price Desc")
px.bar(sdf_price_per_product.limit(10).toPandas(), x="Product_ID", y="Price", title="TOP 10 most expensive Product_IDs")

#### Connection to the event-type
The following plots will represent the event_type distribution within the category_classes, category_codes, category_ids, product_ids and brands.

In [42]:
sdf_category_class_event_distribution = spark.sql("SELECT category_class, \
                                                          event_type, \
                                                          Count(*) AS Count \
                                                    FROM Data \
                                                    GROUP BY category_class, event_type")
#px.sunburst(sdf_category_class_event_distribution.toPandas(), path=['category_class','event_type'], values="Count")

#### Connection to the turnover

In [43]:
 sdf_brand_overview = spark.sql("SELECT brand, \
                                        COUNT(Product_id), \
                                        MEAN(price), \
                                        SUM(viewed_quantity) ,\
                                        SUM(cart_quantity),\
                                        SUM(bougth_quantity), \
                                        SUM(turnover) AS Turnover\
                                FROM Data \
                                GROUP BY brand \
                                ORDER BY Turnover DESC")
                                
px.bar(sdf_brand_overview.limit(10).toPandas(), x='brand', y='Turnover')                             

In [44]:
 sdf_category_code_overview = spark.sql("SELECT category_code, \
                                        COUNT(Product_id), \
                                        MEAN(price), \
                                        SUM(viewed_quantity) ,\
                                        SUM(cart_quantity),\
                                        SUM(bougth_quantity), \
                                        SUM(turnover) AS Turnover\
                                FROM Data \
                                GROUP BY category_code \
                                ORDER BY Turnover DESC")
px.bar(sdf_category_code_overview.limit(10).toPandas(), x='category_code', y='Turnover')

In [45]:
 sdf_category_class_overview = spark.sql("SELECT category_class, \
                                        COUNT(Product_id), \
                                        MEAN(price), \
                                        SUM(viewed_quantity) ,\
                                        SUM(cart_quantity),\
                                        SUM(bougth_quantity), \
                                        SUM(turnover) AS Turnover\
                                FROM Data \
                                GROUP BY category_class \
                                ORDER BY Turnover DESC")
px.bar(sdf_category_class_overview.limit(10).toPandas(), x='category_class', y='Turnover')

### Event_Type and Price
The following plot represents the distribution of the price within the event_type.

In [46]:
px.box(sdf_raw.toPandas(), x="event_type", y="price", title="Price ~ Event_Type")

### User analysis

#### User and turnover

In [47]:
 sdf_user_overview = spark.sql("SELECT user_id, \
                                        COUNT(user_session), \
                                        SUM(viewed_quantity) ,\
                                        SUM(cart_quantity),\
                                        SUM(bougth_quantity), \
                                        SUM(turnover) AS Turnover\
                                FROM Data \
                                GROUP BY user_id \
                                ORDER BY Turnover DESC")
px.bar(sdf_user_overview.limit(10).toPandas(), x='user_id', y='Turnover')

### session analysis



- number of events
- session_start_time
- session_stop_time
- session_success
- products_bought
- products_viewed
- turnover


In [48]:
# different buying behavior between different times (morning more sales than evening?)
sdf_session_raw = sdf.groupBy("user_id", "user_session", "event_type", "product_id", "price").agg(f.max("event_time"), f.min("event_time")).orderBy("user_id", "user_session")

In [49]:
sdf_session_raw.show(30,truncate=False)

+---------+------------------------------------+----------+----------+------+-------------------+-------------------+
|user_id  |user_session                        |event_type|product_id|price |max(event_time)    |min(event_time)    |
+---------+------------------------------------+----------+----------+------+-------------------+-------------------+
|436701163|50de79b1-b0ec-42c0-bbe0-8b5d2dc33e1e|view      |1004856   |128.42|2019-11-01 01:00:23|2019-11-01 01:00:23|
|512367687|09085a31-dc7d-46c1-a0ae-d316569b0b7b|view      |14700422  |168.06|2019-11-12 02:34:42|2019-11-12 02:34:42|
|512370912|daf0bf99-adf1-4877-8308-ae4557b6f01b|view      |6500912   |287.83|2019-11-01 01:00:26|2019-11-01 01:00:26|
|512376444|f7e8ffaa-5103-47f3-90ab-c30f313770c0|view      |12719892  |252.77|2019-11-01 01:00:25|2019-11-01 01:00:25|
|512376444|f7e8ffaa-5103-47f3-90ab-c30f313770c0|view      |12720155  |58.17 |2019-11-01 01:00:42|2019-11-01 01:00:42|
|512376444|f7e8ffaa-5103-47f3-90ab-c30f313770c0|view    

In [50]:
sdf_session = sdf.select("user_id", "user_session", "event_type", "product_id", "price", "event_time").orderBy("user_id", "user_session")

In [51]:
sdf_session.show(30, truncate=False)

+---------+------------------------------------+----------+----------+------+-------------------+
|user_id  |user_session                        |event_type|product_id|price |event_time         |
+---------+------------------------------------+----------+----------+------+-------------------+
|436701163|50de79b1-b0ec-42c0-bbe0-8b5d2dc33e1e|view      |1004856   |128.42|2019-11-01 01:00:23|
|512367687|09085a31-dc7d-46c1-a0ae-d316569b0b7b|view      |14700422  |168.06|2019-11-12 02:34:42|
|512370912|daf0bf99-adf1-4877-8308-ae4557b6f01b|view      |6500912   |287.83|2019-11-01 01:00:26|
|512376444|f7e8ffaa-5103-47f3-90ab-c30f313770c0|view      |12719892  |252.77|2019-11-01 01:00:25|
|512376444|f7e8ffaa-5103-47f3-90ab-c30f313770c0|view      |12719154  |88.81 |2019-11-01 01:00:08|
|512376444|f7e8ffaa-5103-47f3-90ab-c30f313770c0|view      |12720155  |58.17 |2019-11-01 01:00:42|
|512384603|8b1cd19b-980d-440e-887e-0ac7516009c7|view      |6701005   |169.36|2019-11-12 02:34:42|
|512416379|4dfe2c67-

### Customer Profiles

In preparation for clustering a customer profile will be created:

- customer_id
- number_of_view_events
- number_of_cart_events
- number_of_purchase_events
- total_turnover
- number_of_bought_items (resolve multiple purchasing events for quantity)
- avg_sold_cart
- avg_session_time
- avg_actions_per_session



In [52]:
# time period simply dependend on input data

### Customer Journey

- Path to success (purchase)
- Path to failure (no purchase)

### Correlation matrix

#### Daytime - Correlation Matrix

In [53]:
sdf_corr_time = spark.sql("SELECT event_time, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")
sdf_corr_time = sdf_corr_time.withColumn("hour", f.hour("event_time"))

In [54]:
sdf_corr_time = sdf_corr_time.withColumn('Morning', f.when((f.col('hour')>=6) & (f.col('hour')<12), f.lit(1)).otherwise(0))
sdf_corr_time = sdf_corr_time.withColumn('Afternoon', f.when((f.col('hour')>=12) & (f.col('hour')<18), f.lit(1)).otherwise(0))
sdf_corr_time = sdf_corr_time.withColumn('Evening', f.when(f.col('hour') > 18 , f.lit(1)).otherwise(0))
sdf_corr_time = sdf_corr_time.withColumn('Night', f.when(f.col('hour') < 6, f.lit(1)).otherwise(0))

In [55]:
sdf_corr_time = sdf_corr_time.select("Morning", "Afternoon", "Evening", "Night",  "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [56]:
sdf_corr_time.toPandas().corr().style.background_gradient(cmap='bwr')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1264: RuntimeWarning:

All-NaN slice encountered

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1265: RuntimeWarning:

All-NaN slice encountered



,Morning,Afternoon,Evening,Night,turnover,bougth_quantity,viewed_quantity,cart_quantity
Morning,nan,nan,nan,nan,nan,nan,nan,nan
Afternoon,nan,1.000000,nan,-1.000000,-0.015222,-0.016738,0.023769,-0.016738
Evening,nan,nan,nan,nan,nan,nan,nan,nan
Night,nan,-1.000000,nan,1.000000,0.015222,0.016738,-0.023769,0.016738
turnover,nan,-0.015222,nan,0.015222,1.000000,0.909438,-0.640407,-0.007516
bougth_quantity,nan,-0.016738,nan,0.016738,0.909438,1.000000,-0.704179,-0.008264
viewed_quantity,nan,0.023769,nan,-0.023769,-0.640407,-0.704179,1.000000,-0.704179
cart_quantity,nan,-0.016738,nan,0.016738,-0.007516,-0.008264,-0.704179,1.000000


#### Weekday - Correlation Matrix

In [57]:
sdf_corr_dayofweek = spark.sql("SELECT dayofweek, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")

In [58]:
# One-hot-encoding
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Sunday', f.when(f.col('dayofweek') == '1', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Monday', f.when(f.col('dayofweek') == '2', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Tuesday', f.when(f.col('dayofweek') == '3', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Wednesday', f.when(f.col('dayofweek') == '4', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Thursday', f.when(f.col('dayofweek') == '5', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Friday', f.when(f.col('dayofweek') == '6', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Saturday', f.when(f.col('dayofweek') == '7', f.lit(1)).otherwise(0))

In [59]:
sdf_corr_dayofweek = sdf_corr_dayofweek.select("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday",  "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [60]:
sdf_corr_dayofweek.toPandas().corr().style.background_gradient(cmap='bwr')

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,turnover,bougth_quantity,viewed_quantity,cart_quantity
Monday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Tuesday,nan,1.000000,-0.053503,nan,-0.823822,nan,nan,-0.024025,-0.026418,-0.082929,0.143211
Wednesday,nan,-0.053503,1.000000,nan,-0.521960,nan,nan,-0.015222,-0.016738,0.023769,-0.016738
Thursday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Friday,nan,-0.823822,-0.521960,nan,1.000000,nan,nan,0.029163,0.032067,0.057344,-0.112829
Saturday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Sunday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
turnover,nan,-0.024025,-0.015222,nan,0.029163,nan,nan,1.000000,0.909438,-0.640407,-0.007516
bougth_quantity,nan,-0.026418,-0.016738,nan,0.032067,nan,nan,0.909438,1.000000,-0.704179,-0.008264
viewed_quantity,nan,-0.082929,0.023769,nan,0.057344,nan,nan,-0.640407,-0.704179,1.000000,-0.704179


#### Category Class - Correlation Matrix

In [61]:
sdf_corr_category_class = spark.sql("SELECT category_class, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")

In [62]:
# One-hot-encoding
sdf_corr_category_class = sdf_corr_category_class.withColumn('computers', f.when(f.col('category_class') == 'computers', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('auto', f.when(f.col('category_class') == 'auto', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('apparel', f.when(f.col('category_class') == 'apparel', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('appliances', f.when(f.col('category_class') == 'appliances', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('furniture', f.when(f.col('category_class') == 'furniture', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('accessories', f.when(f.col('category_class') == 'accessories', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('electronics', f.when(f.col('category_class') == 'electronics', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('construction', f.when(f.col('category_class') == 'construction', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('not_defined', f.when(f.col('category_class') == 'not defined', f.lit(1)).otherwise(0))

In [63]:
sdf_corr_category_class = sdf_corr_category_class.select("computers", "auto", "apparel", "appliances", "furniture", "accessories", "electronics", "construction", "not_defined", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [64]:
sdf_corr_category_class.toPandas().corr().style.background_gradient(cmap='bwr')

,computers,auto,apparel,appliances,furniture,accessories,electronics,construction,not_defined,turnover,bougth_quantity,viewed_quantity,cart_quantity
computers,1.000000,-0.066120,-0.053646,-0.100958,-0.086472,-0.021674,-0.197201,-0.062207,nan,-0.027934,-0.030715,0.043619,-0.030715
auto,-0.066120,1.000000,-0.031072,-0.058476,-0.050086,-0.012554,-0.114221,-0.036031,nan,-0.016180,-0.017791,0.025265,-0.017791
apparel,-0.053646,-0.031072,1.000000,-0.047444,-0.040636,-0.010186,-0.092672,-0.029233,nan,-0.013127,-0.014434,0.020498,-0.014434
appliances,-0.100958,-0.058476,-0.047444,1.000000,-0.076475,-0.019168,-0.174402,-0.055015,nan,-0.024704,-0.027164,0.038576,-0.027164
furniture,-0.086472,-0.050086,-0.040636,-0.076475,1.000000,-0.016418,-0.149378,-0.047121,nan,0.229308,0.165969,-0.101326,-0.023267
accessories,-0.021674,-0.012554,-0.010186,-0.019168,-0.016418,1.000000,-0.037442,-0.011811,nan,-0.005304,-0.005832,0.008282,-0.005832
electronics,-0.197201,-0.114221,-0.092672,-0.174402,-0.149378,-0.037442,1.000000,-0.107461,nan,0.003459,0.051348,-0.072920,0.051348
construction,-0.062207,-0.036031,-0.029233,-0.055015,-0.047121,-0.011811,-0.107461,1.000000,nan,-0.015222,-0.016738,0.023769,-0.016738
not_defined,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
turnover,-0.027934,-0.016180,-0.013127,-0.024704,0.229308,-0.005304,0.003459,-0.015222,nan,1.000000,0.909438,-0.640407,-0.007516


#### Month - Correlation Matrix

In [65]:
sdf_corr_month = spark.sql("SELECT dayofmonth, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")

In [66]:
# One-hot-encoding
sdf_corr_month = sdf_corr_month.withColumn('Beginningofmonth', f.when(f.col('dayofmonth')<10, f.lit(1)).otherwise(0))
sdf_corr_month = sdf_corr_month.withColumn('Middleofmonth', f.when((f.col('dayofmonth')>=10) & (f.col('dayofmonth')<20), f.lit(1)).otherwise(0))
sdf_corr_month = sdf_corr_month.withColumn('Endofmonth', f.when(f.col('dayofmonth') > 20 , f.lit(1)).otherwise(0))

In [67]:
sdf_corr_month = sdf_corr_month.select("Beginningofmonth", "Middleofmonth", "Endofmonth", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [68]:
sdf_corr_month.toPandas().corr().style.background_gradient(cmap='bwr')

,Beginningofmonth,Middleofmonth,Endofmonth,turnover,bougth_quantity,viewed_quantity,cart_quantity
Beginningofmonth,1.000000,-1.000000,nan,0.024025,0.026418,0.082929,-0.143211
Middleofmonth,-1.000000,1.000000,nan,-0.024025,-0.026418,-0.082929,0.143211
Endofmonth,nan,nan,nan,nan,nan,nan,nan
turnover,0.024025,-0.024025,nan,1.000000,0.909438,-0.640407,-0.007516
bougth_quantity,0.026418,-0.026418,nan,0.909438,1.000000,-0.704179,-0.008264
viewed_quantity,0.082929,-0.082929,nan,-0.640407,-0.704179,1.000000,-0.704179
cart_quantity,-0.143211,0.143211,nan,-0.007516,-0.008264,-0.704179,1.000000


#### Price - Correlation Matrix

In [69]:
sdf_corr_price = spark.sql("SELECT price, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")
sdf_corr_price.toPandas().corr().style.background_gradient(cmap='bwr')

,price,turnover,bougth_quantity,viewed_quantity,cart_quantity
price,1.000000,0.032651,0.018623,-0.014629,0.001980
turnover,0.032651,1.000000,0.909438,-0.640407,-0.007516
bougth_quantity,0.018623,0.909438,1.000000,-0.704179,-0.008264
viewed_quantity,-0.014629,-0.640407,-0.704179,1.000000,-0.704179
cart_quantity,0.001980,-0.007516,-0.008264,-0.704179,1.000000


## Clustering

- Customer classification (h,m,l)
- Customer related content/ recomenadion